In [1]:
import ast
import json
import os
import pickle
import warnings
from collections import Counter
from pathlib import Path
from random import randint, random

import matplotlib.pyplot as plt
import nltk
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras.backend as K
from google.colab import drive
from scipy.sparse import coo_matrix, hstack
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.feature_selection import SelectPercentile, chi2
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics.pairwise import euclidean_distances as ED
from tensorflow import keras
from tqdm.notebook import tqdm

In [2]:
warnings.filterwarnings("ignore")

In [3]:
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Считаем предобработанные данные

In [5]:
DATA_PATH = Path("/content/drive/MyDrive/kion_train/")
users_df = pd.read_csv(DATA_PATH / "users_processed_kion.csv")
items_df = pd.read_csv(DATA_PATH / "items_processed_kion.csv")
interactions_df = pd.read_csv(DATA_PATH / "interactions_processed_kion.csv")

In [6]:
users_df.head()

,user_id,age,income,sex,kids_flg
0,973171,age_25_34,income_60_90,M,True
1,962099,age_18_24,income_20_40,M,False
2,1047345,age_45_54,income_40_60,F,False
3,721985,age_45_54,income_20_40,F,False
4,704055,age_35_44,income_60_90,F,False


In [7]:
items_df = items_df.rename(columns={"id": "item_id"})

In [8]:
items_df.head()

,item_id,content_type,title,title_orig,genres,countries,for_kids,age_rating,studios,directors,actors,description,keywords,release_year_cat
0,10711,film,поговори с ней,Hable con ella,"драмы, зарубежные, детективы, мелодрамы",испания,False,16.0,unknown,педро альмодовар,"Адольфо Фернандес, Ана Фернандес, Дарио Гранди...",Мелодрама легендарного Педро Альмодовара «Пого...,"Поговори, ней, 2002, Испания, друзья, любовь, ...",2000-2010
1,2508,film,голые перцы,Search Party,"зарубежные, приключения, комедии",сша,False,16.0,unknown,скот армстронг,"Адам Палли, Брайан Хаски, Дж.Б. Смув, Джейсон ...",Уморительная современная комедия на популярную...,"Голые, перцы, 2014, США, друзья, свадьбы, прео...",2010-2020
2,10716,film,тактическая сила,Tactical Force,"криминал, зарубежные, триллеры, боевики, комедии",канада,False,16.0,unknown,адам п. калтраро,"Адриан Холмс, Даррен Шалави, Джерри Вассерман,...",Профессиональный рестлер Стив Остин («Все или ...,"Тактическая, сила, 2011, Канада, бандиты, ганг...",2010-2020
3,7868,film,45 лет,45 Years,"драмы, зарубежные, мелодрамы",великобритания,False,16.0,unknown,эндрю хэй,"Александра Риддлстон-Барретт, Джеральдин Джейм...","Шарлотта Рэмплинг, Том Кортни, Джеральдин Джей...","45, лет, 2015, Великобритания, брак, жизнь, лю...",2010-2020
4,16268,film,все решает мгновение,None,"драмы, спорт, советские, мелодрамы",ссср,False,12.0,ленфильм,виктор садовский,"Александр Абдулов, Александр Демьяненко, Алекс...",Расчетливая чаровница из советского кинохита «...,"Все, решает, мгновение, 1978, СССР, сильные, ж...",1970-1980


In [9]:
interactions_df.head()

,user_id,item_id,last_watch_dt,total_dur,watched_pct
0,176549,9506,2021-05-11,4250,72
1,699317,1659,2021-05-29,8317,100
2,656683,7107,2021-05-09,10,0
3,864613,7638,2021-07-05,14483,100
4,964868,9506,2021-04-30,6725,100


## Готовим фичи пользователей

Посмотрим, какие фичи в датасете фильмов являются категориальными и закодируем их с помощью one-hot encoding.

In [10]:
user_cat_feats = ["age", "income", "sex", "kids_flg"]
# из исходного датафрейма оставим только item_id - этот признак нам понадобится позже
# для того, чтобы маппить айтемы из датафрейма с фильмами с айтемами
# из датафрейма с взаимодействиями
users_ohe_df = users_df.user_id
for feat in user_cat_feats:
    # получаем датафрейм с one-hot encoding для каждой категориальной фичи
    ohe_feat_df = pd.get_dummies(users_df[feat], prefix=feat)
    # конкатенируем ohe-hot датафрейм с датафреймом,
    # который мы получили на предыдущем шаге
    users_ohe_df = pd.concat([users_ohe_df, ohe_feat_df], axis=1)

users_ohe_df.head()

,user_id,age_age_18_24,age_age_25_34,age_age_35_44,age_age_45_54,age_age_55_64,age_age_65_inf,age_age_unknown,income_income_0_20,income_income_150_inf,income_income_20_40,income_income_40_60,income_income_60_90,income_income_90_150,income_income_unknown,sex_F,sex_M,sex_sex_unknown,kids_flg_False,kids_flg_True
0,973171,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1
1,962099,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0
2,1047345,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0
3,721985,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0
4,704055,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0


## Готовим фичи айтемов

Кодируем их точно так же - one-hot'ом.

In [11]:
items_df.head()

,item_id,content_type,title,title_orig,genres,countries,for_kids,age_rating,studios,directors,actors,description,keywords,release_year_cat
0,10711,film,поговори с ней,Hable con ella,"драмы, зарубежные, детективы, мелодрамы",испания,False,16.0,unknown,педро альмодовар,"Адольфо Фернандес, Ана Фернандес, Дарио Гранди...",Мелодрама легендарного Педро Альмодовара «Пого...,"Поговори, ней, 2002, Испания, друзья, любовь, ...",2000-2010
1,2508,film,голые перцы,Search Party,"зарубежные, приключения, комедии",сша,False,16.0,unknown,скот армстронг,"Адам Палли, Брайан Хаски, Дж.Б. Смув, Джейсон ...",Уморительная современная комедия на популярную...,"Голые, перцы, 2014, США, друзья, свадьбы, прео...",2010-2020
2,10716,film,тактическая сила,Tactical Force,"криминал, зарубежные, триллеры, боевики, комедии",канада,False,16.0,unknown,адам п. калтраро,"Адриан Холмс, Даррен Шалави, Джерри Вассерман,...",Профессиональный рестлер Стив Остин («Все или ...,"Тактическая, сила, 2011, Канада, бандиты, ганг...",2010-2020
3,7868,film,45 лет,45 Years,"драмы, зарубежные, мелодрамы",великобритания,False,16.0,unknown,эндрю хэй,"Александра Риддлстон-Барретт, Джеральдин Джейм...","Шарлотта Рэмплинг, Том Кортни, Джеральдин Джей...","45, лет, 2015, Великобритания, брак, жизнь, лю...",2010-2020
4,16268,film,все решает мгновение,None,"драмы, спорт, советские, мелодрамы",ссср,False,12.0,ленфильм,виктор садовский,"Александр Абдулов, Александр Демьяненко, Алекс...",Расчетливая чаровница из советского кинохита «...,"Все, решает, мгновение, 1978, СССР, сильные, ж...",1970-1980


In [12]:
item_cat_feats = [
    "content_type",
    "release_year_cat",
    "for_kids",
    "age_rating",
    "studios",
    "countries",
    "directors",
]

items_ohe_df = items_df.item_id

for feat in item_cat_feats:
    ohe_feat_df = pd.get_dummies(items_df[feat], prefix=feat)
    items_ohe_df = pd.concat([items_ohe_df, ohe_feat_df], axis=1)

items_ohe_df.head()

,item_id,content_type_film,content_type_series,release_year_cat_1920-1930,release_year_cat_1930-1940,release_year_cat_1940-1950,release_year_cat_1950-1960,release_year_cat_1960-1970,release_year_cat_1970-1980,release_year_cat_1980-1990,...,directors_ярив хоровиц,directors_ярон зильберман,directors_ярополк лапшин,directors_ярослав лупий,"directors_ярроу чейни, скотт моужер",directors_ясина сезар,directors_ясуоми умэцу,"directors_ёдзи фукуяма, ацуко фукусима, николас де креси, синъитиро ватанабэ, сёдзи кавамори",directors_ёлкин туйчиев,directors_ён сан-хо
0,10711,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2508,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,10716,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,7868,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,16268,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


## Сделаем матрицу взаимодействий

В датасете взаимодействий есть непопулярные фильмы и малоактивные пользователи. Кроме того, в таблице взаимодействий есть события с низким качеством взаимодействия - когда юзер начал смотреть фильм, но вскоре после начала просмотра выключил.

Отфильтруем такие события*, малоактивных юзеров и непопулярные фильмы.


In [13]:
print(f"N users before: {interactions_df.user_id.nunique()}")
print(f"N items before: {interactions_df.item_id.nunique()}\n")

# отфильтруем все события взаимодействий, в которых пользователь посмотрел
# фильм менее чем на 10 процентов
interactions_df = interactions_df[interactions_df.watched_pct > 10]

# соберем всех пользователей, которые посмотрели
# больше 10 фильмов (можете выбрать другой порог)
valid_users = []

c = Counter(interactions_df.user_id)
for user_id, entries in c.most_common():
    if entries > 10:
        valid_users.append(user_id)

# и соберем все фильмы, которые посмотрели больше 10 пользователей
valid_items = []

c = Counter(interactions_df.item_id)
for item_id, entries in c.most_common():
    if entries > 10:
        valid_items.append(item_id)

# отбросим непопулярные фильмы и неактивных юзеров
interactions_df = interactions_df[interactions_df.user_id.isin(valid_users)]
interactions_df = interactions_df[interactions_df.item_id.isin(valid_items)]

print(f"N users after: {interactions_df.user_id.nunique()}")
print(f"N items after: {interactions_df.item_id.nunique()}")

N users before: 962179
N items before: 15706

N users after: 79515
N items after: 6901


После фильтрации может получиться так, что некоторые айтемы/юзеры есть в датасете взаимодействий, но при этом они отсутствуют в датасетах айтемов/юзеров или наоборот. Поэтому найдем id айтемов и id юзеров, которые есть во всех датасетах и оставим только их.

In [21]:
common_users = set(interactions_df.user_id.unique()).intersection(
    set(users_ohe_df.user_id.unique())
)
common_items = set(interactions_df.item_id.unique()).intersection(
    set(items_ohe_df.item_id.unique())
)

print(len(common_users))
print(len(common_items))

interactions_df = interactions_df[interactions_df.item_id.isin(common_items)]
interactions_df = interactions_df[interactions_df.user_id.isin(common_users)]

items_ohe_df = items_ohe_df[items_ohe_df.item_id.isin(common_items)]
users_ohe_df = users_ohe_df[users_ohe_df.user_id.isin(common_users)]

65974
6897


Соберем взаимодействия в матрицу user*item так, чтобы в строках этой матрицы были user_id, в столбцах - item_id, а на пересечениях строк и столбцов - единица, если пользователь взаимодействовал с айтемом и ноль, если нет.

Такую матрицу удобно собирать в numpy array, однако нужно помнить, что numpy array индексируется порядковыми индексами, а нам же удобнее использовать item_id и user_id.

Создадим некие внутренние индексы для user_id и item_id - uid и iid. Для этого просто соберем все user_id и item_id и пронумеруем их по порядку.

In [15]:
interactions_df["uid"] = interactions_df["user_id"].astype("category")
interactions_df["uid"] = interactions_df["uid"].cat.codes

interactions_df["iid"] = interactions_df["item_id"].astype("category")
interactions_df["iid"] = interactions_df["iid"].cat.codes

print(sorted(interactions_df.iid.unique())[:5])
print(sorted(interactions_df.uid.unique())[:5])
interactions_df.head()

[0, 1, 2, 3, 4]
[0, 1, 2, 3, 4]


,user_id,item_id,last_watch_dt,total_dur,watched_pct,uid,iid
0,176549,9506,2021-05-11,4250,72,10616,3944
1,699317,1659,2021-05-29,8317,100,42131,675
6,1016458,354,2021-08-14,1672,25,61024,139
7,884009,693,2021-08-04,703,14,53150,279
14,5324,8437,2021-04-18,6598,92,310,3485


Отнормируем матрицу взаимодействий

In [16]:
interactions_vec = np.zeros((interactions_df.uid.nunique(), interactions_df.iid.nunique()))

# Используем информацию о качестве взаимодействия юзеров с айтемами для более репрезентативного сэмплирования
for user_id, item_id, weight in zip(
    interactions_df.uid, interactions_df.iid, interactions_df.watched_pct
):
    interactions_vec[user_id, item_id] += weight / 100


res = interactions_vec.sum(axis=1)
for i in range(len(interactions_vec)):
    interactions_vec[i] /= res[i]

In [20]:
print(interactions_df.item_id.nunique())
print(items_ohe_df.item_id.nunique())
print(interactions_df.user_id.nunique())
print(users_ohe_df.user_id.nunique())

print(set(items_ohe_df.item_id.unique()) - set(interactions_df.item_id.unique()))

6897
6901
65974
65974
{11805, 9788, 11501, 1734}


Для того, чтобы можно было удобно превратить iid/uid в item_id/user_id и наоборот соберем словари

{iid: item_id}, {uid: user_id} и {item_id: iid}, {user_id: uid}.

In [18]:
iid_to_item_id = (
    interactions_df[["iid", "item_id"]].drop_duplicates().set_index("iid").to_dict()["item_id"]
)
item_id_to_iid = (
    interactions_df[["iid", "item_id"]].drop_duplicates().set_index("item_id").to_dict()["iid"]
)

uid_to_user_id = (
    interactions_df[["uid", "user_id"]].drop_duplicates().set_index("uid").to_dict()["user_id"]
)
user_id_to_uid = (
    interactions_df[["uid", "user_id"]].drop_duplicates().set_index("user_id").to_dict()["uid"]
)

И проиндексируем датасеты users_ohe_df и items_ohe_df по внутренним айди:

In [22]:
items_ohe_df["iid"] = items_ohe_df["item_id"].apply(lambda x: item_id_to_iid[x])
items_ohe_df = items_ohe_df.set_index("iid")

users_ohe_df["uid"] = users_ohe_df["user_id"].apply(lambda x: user_id_to_uid[x])
users_ohe_df = users_ohe_df.set_index("uid")

In [23]:
def triplet_loss(y_true, y_pred, n_dims=128, alpha=0.4):
    # будем ожидать, что на вход функции прилетит три сконкатенированных
    # вектора - вектор юзера и два вектора айтема
    anchor = y_pred[:, 0:n_dims]
    positive = y_pred[:, n_dims : n_dims * 2]
    negative = y_pred[:, n_dims * 2 : n_dims * 3]

    # считаем расстояния от вектора юзера до вектора хорошего айтема
    pos_dist = K.sum(K.square(anchor - positive), axis=1)
    # и до плохого
    neg_dist = K.sum(K.square(anchor - negative), axis=1)

    # считаем лосс
    basic_loss = pos_dist - neg_dist + alpha
    loss = K.maximum(basic_loss, 0.0)  # возвращаем ноль, если лосс отрицательный

    return loss

## Добавим текстовые признаки

In [24]:
# представляет текст в виде мешка слов
count_vectorizer = CountVectorizer(lowercase=True, tokenizer=nltk.word_tokenize)
# преобразует текст в набор признаков TF-IDF
tf_idf_transformer = TfidfTransformer()
# отбирает 1% признаков, наиболее важных для классификации при критерию хи-квадрат
selector = SelectPercentile(chi2, percentile=1)

In [25]:
popular_items = (
    interactions_df.groupby("item_id")
    .agg({"user_id": "count"})
    .rename(columns={"user_id": "popularity"})
)

In [26]:
items_df = items_df.merge(popular_items, on="item_id")

In [27]:
bow = count_vectorizer.fit_transform(items_df["description"])
tf_idf = tf_idf_transformer.fit_transform(bow)
tf_idf

<6897x71113 sparse matrix of type '<class 'numpy.float64'>'
	with 435074 stored elements in Compressed Sparse Row format>

In [ ]:
# отбираем признаки, наиболее влияющие на популярность фильма. 
# наверное, так может появиться bias на популярность, но не придумала, что еще можно использовать в качестве таргета для отбора
selected_features = selector.fit_transform(tf_idf, items_df["popularity"])

In [29]:
text_features = pd.DataFrame(selected_features.toarray(), columns=selector.get_feature_names_out())

In [30]:
text_features["item_id"] = items_df["item_id"]
items_ohe_df = items_ohe_df.merge(text_features, on="item_id", how="left")
items_ohe_df = items_ohe_df.fillna(0)

## Генератор и семплирование

Сделаем простой генератор. Он будет брать рандромного юзера, и два разных айтема - хороший пример и плохой:
- хорошим примером будет тот айтем, который был взят из датасета взаимодействий в соответствии с распределением просмотренных айтемов для этого юзера;
- а плохим айтемом будет просто любой другой _случайный айтем_*


In [31]:
def generator(items, users, interactions, batch_size=1024):
    while True:
        uid_meta = []
        uid_interaction = []
        pos = []
        neg = []
        for _ in range(batch_size):
            # берем рандомный uid
            uid_i = randint(0, interactions.shape[0] - 1)
            # id хорошего айтема
            pos_i = np.random.choice(range(interactions.shape[1]), p=interactions[uid_i])
            # id плохого айтема
            neg_i = np.random.choice(range(interactions.shape[1]))
            # фичи юзера
            uid_meta.append(users.iloc[uid_i])
            # вектор айтемов, с которыми юзер взаимодействовал
            uid_interaction.append(interactions_vec[uid_i])
            # фичи хорошего айтема
            pos.append(items.iloc[pos_i])
            # фичи плохого айтема
            neg.append(items.iloc[neg_i])

        yield [np.array(uid_meta), np.array(uid_interaction), np.array(pos), np.array(neg)], [
            np.array(uid_meta),
            np.array(uid_interaction),
        ]

In [32]:
# инициализируем генератор
gen = generator(
    items=items_ohe_df.drop(["item_id"], axis=1),
    users=users_ohe_df.drop(["user_id"], axis=1),
    interactions=interactions_vec,
)

ret = next(gen)


print(f"вектор фичей юзера: {ret[0][0].shape}")
print(f"вектор взаимодействий юзера с айтемами: {ret[0][1].shape}")
print(f"вектор 'хорошего' айтема: {ret[0][2].shape}")
print(f"вектор 'плохого' айтема: {ret[0][3].shape}")
print()
print(f"вектор фичей юзера: {ret[1][0].shape}")
print(f"вектор взаимодействий юзера с айтемами: {ret[1][1].shape}")

вектор фичей юзера: (1024, 19)
вектор взаимодействий юзера с айтемами: (1024, 6897)
вектор 'хорошего' айтема: (1024, 9299)
вектор 'плохого' айтема: (1024, 9299)

вектор фичей юзера: (1024, 19)
вектор взаимодействий юзера с айтемами: (1024, 6897)


## Создание модели

In [33]:
N_FACTORS = 128

# в датасетах есть столбец user_id/item_id, помним, что он не является фичей для обучения!
ITEM_MODEL_SHAPE = (items_ohe_df.drop(["item_id"], axis=1).shape[1],)
USER_META_MODEL_SHAPE = (users_ohe_df.drop(["user_id"], axis=1).shape[1],)

USER_INTERACTION_MODEL_SHAPE = (interactions_vec.shape[1],)

print(f"N_FACTORS: {N_FACTORS}")
print(f"ITEM_MODEL_SHAPE: {ITEM_MODEL_SHAPE}")
print(f"USER_META_MODEL_SHAPE: {USER_META_MODEL_SHAPE}")
print(f"USER_INTERACTION_MODEL_SHAPE: {USER_INTERACTION_MODEL_SHAPE}")

N_FACTORS: 128
ITEM_MODEL_SHAPE: (9299,)
USER_META_MODEL_SHAPE: (19,)
USER_INTERACTION_MODEL_SHAPE: (6897,)


In [34]:
def item_model(n_factors=N_FACTORS):
    # входной слой
    inp = keras.layers.Input(shape=ITEM_MODEL_SHAPE)

    # полносвязный слой
    layer_1 = keras.layers.Dense(
        N_FACTORS,
        activation="elu",
        use_bias=False,
        kernel_regularizer=keras.regularizers.l2(1e-6),
        activity_regularizer=keras.regularizers.l2(l2=1e-6),
    )(inp)

    # делаем residual connection - складываем два слоя,
    # чтобы градиенты не затухали во время обучения
    layer_2 = keras.layers.Dense(
        N_FACTORS,
        activation="elu",
        use_bias=False,
        kernel_regularizer=keras.regularizers.l2(1e-6),
        activity_regularizer=keras.regularizers.l2(l2=1e-6),
    )(layer_1)

    add = keras.layers.Add()([layer_1, layer_2])

    # выходной слой
    out = keras.layers.Dense(
        N_FACTORS,
        activation="linear",
        use_bias=False,
        kernel_regularizer=keras.regularizers.l2(1e-6),
        activity_regularizer=keras.regularizers.l2(l2=1e-6),
    )(add)

    return keras.models.Model(inp, out)


def user_model(n_factors=N_FACTORS):
    # входной слой для вектора фичей юзера (из users_ohe_df)
    inp_meta = keras.layers.Input(shape=USER_META_MODEL_SHAPE)
    # входной слой для вектора просмотров (из iteractions_vec)
    inp_interaction = keras.layers.Input(shape=USER_INTERACTION_MODEL_SHAPE)

    # полносвязный слой
    layer_1_meta = keras.layers.Dense(
        N_FACTORS,
        activation="elu",
        use_bias=False,
        kernel_regularizer=keras.regularizers.l2(1e-6),
        activity_regularizer=keras.regularizers.l2(l2=1e-6),
    )(inp_meta)

    layer_1_interaction = keras.layers.Dense(
        N_FACTORS,
        activation="elu",
        use_bias=False,
        kernel_regularizer=keras.regularizers.l2(1e-6),
        activity_regularizer=keras.regularizers.l2(l2=1e-6),
    )(inp_interaction)

    # делаем residual connection - складываем два слоя,
    # чтобы градиенты не затухали во время обучения
    layer_2_meta = keras.layers.Dense(
        N_FACTORS,
        activation="elu",
        use_bias=False,
        kernel_regularizer=keras.regularizers.l2(1e-6),
        activity_regularizer=keras.regularizers.l2(l2=1e-6),
    )(layer_1_meta)

    add = keras.layers.Add()([layer_1_meta, layer_2_meta])

    # конкатенируем вектор фичей с вектором просмотров
    concat_meta_interaction = keras.layers.Concatenate()([add, layer_1_interaction])

    # выходной слой
    out = keras.layers.Dense(
        N_FACTORS,
        activation="linear",
        use_bias=False,
        kernel_regularizer=keras.regularizers.l2(1e-6),
        activity_regularizer=keras.regularizers.l2(l2=1e-6),
    )(concat_meta_interaction)

    return keras.models.Model([inp_meta, inp_interaction], out)


# инициализируем модели юзера и айтема
i2v = item_model()
u2v = user_model()

# вход для вектора фичей юзера (из users_ohe_df)
ancor_meta_in = keras.layers.Input(shape=USER_META_MODEL_SHAPE)
# вход для вектора просмотра юзера (из interactions_vec)
ancor_interaction_in = keras.layers.Input(shape=USER_INTERACTION_MODEL_SHAPE)

# вход для вектора "хорошего" айтема
pos_in = keras.layers.Input(shape=ITEM_MODEL_SHAPE)
# вход для вектора "плохого" айтема
neg_in = keras.layers.Input(shape=ITEM_MODEL_SHAPE)

# получаем вектор юзера
ancor = u2v([ancor_meta_in, ancor_interaction_in])
# получаем вектор "хорошего" айтема
pos = i2v(pos_in)
# получаем вектор "плохого" айтема
neg = i2v(neg_in)

# конкатенируем полученные векторы
res = keras.layers.Concatenate(name="concat_ancor_pos_neg")([ancor, pos, neg])

# собираем модель
model = keras.models.Model([ancor_meta_in, ancor_interaction_in, pos_in, neg_in], res)

In [35]:
model_name = "recsys_resnet_linear"

# логируем процесс обучения в тензорборд
t_board = keras.callbacks.TensorBoard(log_dir=f"runs/{model_name}")

# уменьшаем learning_rate, если лосс долго не уменьшается (в течение двух эпох)
decay = keras.callbacks.ReduceLROnPlateau(monitor="loss", patience=2, factor=0.8, verbose=1)

# сохраняем модель после каждой эпохи, если лосс уменьшился
check = keras.callbacks.ModelCheckpoint(
    filepath=model_name + "/epoch{epoch}-{loss:.2f}.h5", monitor="loss"
)

In [36]:
# компилируем модель, используем оптимайзер Adam и triplet loss
opt = keras.optimizers.Adam(learning_rate=0.001)
model.compile(loss=triplet_loss, optimizer=opt)

In [37]:
# модель айтема
item_model().summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_8 (InputLayer)        [(None, 9299)]               0         []                            
                                                                                                  
 dense_7 (Dense)             (None, 128)                  1190272   ['input_8[0][0]']             
                                                                                                  
 dense_8 (Dense)             (None, 128)                  16384     ['dense_7[0][0]']             
                                                                                                  
 add_2 (Add)                 (None, 128)                  0         ['dense_7[0][0]',             
                                                                     'dense_8[0][0]']       

In [38]:
# модель юзера
user_model().summary()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_9 (InputLayer)        [(None, 19)]                 0         []                            
                                                                                                  
 dense_10 (Dense)            (None, 128)                  2432      ['input_9[0][0]']             
                                                                                                  
 dense_12 (Dense)            (None, 128)                  16384     ['dense_10[0][0]']            
                                                                                                  
 input_10 (InputLayer)       [(None, 6897)]               0         []                            
                                                                                            

In [39]:
# общая модель
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_4 (InputLayer)        [(None, 19)]                 0         []                            
                                                                                                  
 input_5 (InputLayer)        [(None, 6897)]               0         []                            
                                                                                                  
 input_6 (InputLayer)        [(None, 9299)]               0         []                            
                                                                                                  
 input_7 (InputLayer)        [(None, 9299)]               0         []                            
                                                                                            

# Обучение

In [40]:
# начинаем обучение, не забывая дропнуть столбцы item_id и user_id
# из датафреймов при инициализации генератора.

# batch_size можно (и лучше) поставить побольше, если вы не органичены в ресурсах

model.fit(
    generator(
        items=items_ohe_df.drop(["item_id"], axis=1),
        users=users_ohe_df.drop(["user_id"], axis=1),
        interactions=interactions_vec,
        batch_size=64,
    ),
    steps_per_epoch=100,
    epochs=30,
    initial_epoch=0,
    callbacks=[decay, t_board, check],
)

Epoch 1/30
100/100 [==============================] - 126s 1s/step - loss: 0.3041 - lr: 0.0010
Epoch 2/30
100/100 [==============================] - 123s 1s/step - loss: 0.2196 - lr: 0.0010
Epoch 3/30
100/100 [==============================] - 122s 1s/step - loss: 0.1946 - lr: 0.0010
Epoch 4/30
100/100 [==============================] - 121s 1s/step - loss: 0.1856 - lr: 0.0010
Epoch 5/30
100/100 [==============================] - 121s 1s/step - loss: 0.1675 - lr: 0.0010
Epoch 6/30
100/100 [==============================] - 121s 1s/step - loss: 0.1751 - lr: 0.0010
Epoch 7/30
100/100 [==============================] - ETA: 0s - loss: 0.1683
Epoch 7: ReduceLROnPlateau reducing learning rate to 0.000800000037997961.
100/100 [==============================] - 124s 1s/step - loss: 0.1683 - lr: 0.0010
Epoch 8/30
100/100 [==============================] - 122s 1s/step - loss: 0.1625 - lr: 8.0000e-04
Epoch 9/30
100/100 [==============================] - 121s 1s/step - loss: 0.1570 - lr: 8.0000e

### Пример

In [93]:
# берем рандомного юзера
rand_uid = np.random.choice(list(users_ohe_df.index))

# получаем фичи юзера и вектор его просмотров айтемов
user_meta_feats = users_ohe_df.drop(["user_id"], axis=1).iloc[rand_uid]
user_interaction_vec = interactions_vec[rand_uid]

# берем рандомный айтем
rand_iid = np.random.choice(list(items_ohe_df.index))
# получаем фичи айтема
item_feats = items_ohe_df.drop(["item_id"], axis=1).iloc[rand_iid]

# получаем вектор юзера
user_vec = u2v.predict(
    [np.array(user_meta_feats).reshape(1, -1), np.array(user_interaction_vec).reshape(1, -1)]
)

# и вектор айтема
item_vec = i2v.predict(np.array(item_feats).reshape(1, -1))

# считаем расстояние между вектором юзера и вектором айтема
ED(user_vec, item_vec)

1/1 [==============================] - 0s 23ms/step


array([[1.4940156]], dtype=float32)

# Готовим рекомендации для сервиса

In [94]:
# получаем фичи всех айтемов
items_feats = items_ohe_df.drop(["item_id"], axis=1).to_numpy()
# получаем векторы всех айтемов
items_vecs = i2v.predict(items_feats)

# считаем расстояния
dists = ED(user_vec, items_vecs)

165/165 [==============================] - 1s 5ms/step


In [88]:
def recommend_to_user(user_id, items_vecs, topn=10):
    uid = user_id_to_uid[user_id]
    user_meta_feats = users_ohe_df.drop(["user_id"], axis=1).iloc[uid]
    user_interaction_vec = interactions_vec[uid]
    user_vec = u2v.predict(
        [np.array(user_meta_feats).reshape(1, -1), np.array(user_interaction_vec).reshape(1, -1)],
        verbose=False,
    )
    dists = ED(user_vec, items_vecs)
    topn_iids = np.argsort(dists, axis=1)[:, :topn]
    topn_iids_item = [iid_to_item_id[iid] for iid in topn_iids.reshape(-1)]
    return topn_iids_item

In [ ]:
recos = {}
users = list(user_id_to_uid.keys())
for user_id in tqdm(users):
    recos_for_user = recommend_to_user(user_id, items_vecs)
    recos.update({user_id: recos_for_user})

  0%|          | 0/65094 [00:00<?, ?it/s]

К сожалению, получить рекомендации не получилось, т.к. модель оказалась требовательна к оперативной памяти. Пробовала несколько раз, ппроцесс постоянно останавливался